In [ ]:
#!import config/Settings.cs

// Import Azure OpenAI
#r "nuget: Azure.AI.OpenAI, 2.0.0"
using OpenAI.Chat;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new System.ClientModel.ApiKeyCredential(apiKey);
var azureClient = new Azure.AI.OpenAI.AzureOpenAIClient(endpoint, credentials);
var chatClient = azureClient.GetChatClient("gpt-4o");

In [ ]:
var system = 
""" 
Você é um assitente para classificar os sentimentos de um texto.
""";

In [ ]:
var tool = OpenAI.Chat.ChatTool.CreateFunctionTool(
    functionName: "reportSentiment",
    functionDescription: "report the sentiment of the supplied text, and if it have sarcasm in it",
    functionParameters: BinaryData.FromObjectAsJson(
            new
            {
                Type = "object",
                Properties = new
                {
                    Sentiment = new
                    {
                        Type = "string",
                        Description = "The sentiment of the text (positive, negative, neutral)",
                        num = new[] { "positive", "neutral", "negative" },
                    },
                    Sarcasm = new
                    {
                        Type = "boolean",
                        Description = "If the text contains sarcasm",
                    }
                },
                Required = new[] { "sentiment", "sarcasm" },
            },
            new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }
            )
    );



In [ ]:
OpenAI.Chat.ChatCompletionOptions completionOptions = new()
{
    MaxOutputTokenCount=400,
    Temperature=0f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    TopP = 0.95f,
    Tools = { tool }
};



In [ ]:
var messages = new List<OpenAI.Chat.ChatMessage>();

messages.Add(new OpenAI.Chat.SystemChatMessage(system));


In [ ]:
messages.Add(new OpenAI.Chat.UserChatMessage("O restaurante é muito bom, mas a comida é ruim."));
OpenAI.Chat.ChatCompletion response = chatClient.CompleteChat(messages, completionOptions);
response


In [ ]:
messages.Remove(messages.Last());
messages.Add(new OpenAI.Chat.UserChatMessage("O restaurante é tão chique que até as baratas usam gravata."));
OpenAI.Chat.ChatCompletion response = chatClient.CompleteChat(messages, completionOptions);
response
